In [6]:
"""
Hurricane Tracker with NHC Data
===============================

By: Aodhan Sweeney

This program is a recreation of the 2014 hur_tracker.py
originally written by Unidata Intern Florita Rodriguez. The
2019 version comes with updated interface and functionality,
as well as changing certain dependencies.

"""
import os

from datetime import datetime

import numpy as np
import pandas as pd
from pandas import DataFrame
import requests
import cartopy.crs as ccrs
import ipywidgets as widgets
import matplotlib.pyplot as plt


class NHCD():
    """
    Read data from the National Hurricane Center Database (NHCD).

    This class reads and then makes dataframes to easier access NHC Data.

    """

    def __init__(self):
        """
        Create with member attributes and storm info.

        This initiation creates a file table based on a url for all storms in the
        NHCD and puts them into a pandas dataframe. This dataframe is then turned
        into a member atribute '.storm_table'.

        """

        storm_list_columns = ['Name', 'Basin', 'CycloneNum', 'Year', 'StormType', 'Filename']
        file_table = pd.read_csv('http://ftp.nhc.noaa.gov/atcf/index/storm_list.txt',
                                 names=storm_list_columns, header=None, index_col=False,
                                 usecols= [0, 1, 7, 8, 9, 20])
        file_table.Filename = file_table.Filename.str.lower()
        self.storm_table = file_table

    def get_tracks(self, year, filename):
        """
        Make url and pulls track data for a given storm.

        The Url is made by using both the year and the filename. This function will then
        read the data and create a data frame for both the forecast and best tracks and
        compile these data frames into a dictionary. This function returns this dictionary
        of forecast and best track.

        Parameters
        ----------
        self:
            storing the storm dictionary as member attrubute of NHCD
        year: int
            year of the storm incident
        filename: str
            unique filename of the storm which is used for indexing purposes and id
            in the NHCD. The first character is defaulted as space in NHCD so it is clipped
            when being used.

        Returns
        -------
        unique_models: list
            all the models that have run forecasts for this storm throughout its life

        """

        today = datetime.today()
        current_year = today.year
        data_dictionary = {}
        # Current year data is stored in a different location
        if year == str(current_year):
            unformatted_forecast_url = 'http://ftp.nhc.noaa.gov/atcf/aid_public/a{}.dat.gz'
            urlf = unformatted_forecast_url.format(filename[1:])
            unformatted_best_url = 'http://ftp.nhc.noaa.gov/atcf/btk/b{}.dat'
            urlb = unformatted_best_url.format(filename[1:])
        else:
            unformatted_forecast_url = 'http://ftp.nhc.noaa.gov/atcf/archive/{}/a{}.dat.gz'
            urlf = unformatted_forecast_url.format(year, filename[1:])
            unformatted_best_url = 'http://ftp.nhc.noaa.gov/atcf/archive/{}/b{}.dat.gz'
            urlb = unformatted_best_url.format(year, filename[1:])

        url_links = [urlf, urlb]
        url_count = 0
        for url in url_links:
            # Checking if url is valid, if status_code is 200 then website is active
            if requests.get(url).status_code == 200:
                # Creating column names
                storm_data_column_names = ['Basin', 'CycloneNum', 'WarnDT', 'Model',
                                           'Forecast_hour', 'Lat', 'Lon']
                # Create a pandas dataframe using specific columns for a storm
                single_storm = pd.read_csv(url, header=None, names=storm_data_column_names,
                                           index_col=False, usecols= [0, 1, 2, 4, 5, 6, 7])

                # Must convert lats and lons from string to float and preform division by 10
                storm_lats = single_storm['Lat']
                storm_lats = (storm_lats.str.slice(stop=-1))
                storm_lats = storm_lats.astype(float)
                storm_lats = storm_lats/10
                single_storm['Lat'] = storm_lats

                storm_lons = single_storm['Lon']
                storm_lons = (storm_lons.str.slice(stop=-1))
                storm_lons = storm_lons.astype(float)
                storm_lons = -storm_lons/10
                single_storm['Lon'] = storm_lons

                # Change WarnDT to a string
                single_storm['WarnDT'] = [str(x) for x in single_storm['WarnDT']]

                # Adding this newly created DataFrame to a dictionary
                if url_count == 0:
                    data_dictionary['forecast'] = single_storm
                else:
                    data_dictionary['best_track'] = single_storm

            else:
                raise('url {} was not valid, select different storm.'.format(url))

            url_count += 1
        # Turn data_dictionary into a member attribute
        self.storm_dictionary = data_dictionary
        forecast = data_dictionary.get('forecast')
        unique_models, unique_index = list(np.unique(forecast['Model'].values,
                                           return_index=True))
        return(unique_models)

    def model_selection_latlon(self, models):
        """
        Select model type and get lat/lons and track evolution data.

        Parameters
        ----------
        self:
            using storm dictionary attribute and also storing other model_table attribute
            and date_times attribute
        models: list
            unique models that are ran for a storm

        Returns
        -------
        self.model_table: list attribute
            all model forecasts for that specific model type that have been run for a given
            storm

        """
        # We will always plot best track, and thus must save the coordinates for plotting
        best_track = self.storm_dictionary.get('best_track')
        self.date_times = best_track['WarnDT']


        lats = best_track['Lat']
        lons = best_track['Lon']
        self.best_track_coordinates = [lats, lons]

        model_tracks = self.storm_dictionary.get('forecast')

        self.model_table = []
        for model in models:
            one_model_table = model_tracks[model_tracks['Model'] == model]
            self.model_table.append(one_model_table)

        return self.model_table


class NHC_GUI:
    """
    Graphic User Interface designed to allow users to access National Hurricane Center data.

    This class uses ipython widgets, and the order in which the functions appear in this script
    correspond to the order in which the functions and widgets are called/used.
    """

    def __init__(self):
        """
        Create object that references NHC.py and thus the National Hurricane Center.

        This initiation creates the National Hurricane Center object and also creates
        a widget that allows the user to 1.) select the year in which to find storms,
        and 2.) indicate when they have chosen the year and when to continue with parsing
        the storm_table.
        """
        self.NHCD = NHCD()
        self.storm_table = self.NHCD.storm_table
        # Year Slider Widget to select year for which to retrieve storm data.
        self.year_slider = widgets.IntSlider(min=1851, max=2019, value=2019,
                                             description='Storm Year: ')
        widgets.interact(self.get_storms_slider, year_slider=self.year_slider)
        # Storm Track toggle button to initiate storm track retrieval.
        self.track_button = widgets.ToggleButton(value=False, description='Get Storm Tracks',
                                                 disabled=False, button_style='info',
                                                 tooltip='Description')
        widgets.interact(self.get_track, track_button=self.track_button)


    def get_storms_slider(self, year_slider):
        """
        Take a year chosen by the user, and create a list of all storms during that year.

        Parameters
        ----------
        year_slider: ipywidget
            tells value of year chosen by the user
        """
        self.year = str(year_slider)
        self.one_year_table = self.storm_table[self.storm_table.Year == year_slider]
        self.storm_names = widgets.Dropdown(options=self.one_year_table['Name'],
                                            description='Storm Names: ')
        widgets.interact(self.get_name_dropdown, storm_names=self.storm_names)

    def get_name_dropdown(self, storm_names):
        """
        Take names created by previous function and allow selection of which models to plot.

        Parameters
        ----------
        storm_names: list
            all storms in a given year
        """
        name = self.storm_names.value
        one_storm_row = self.one_year_table[self.one_year_table.Name == name]
        self.filename = one_storm_row.Filename
        file_name = self.filename.tolist()
        if self.filename.empty is False:
            self.filename = file_name[0]
        elif self.filename.empty is True:
            raise Exception('ValueError: No file name data for this year.')

    def get_track(self, track_button):
        """
        Query whether track button has been toggled, and create select model widget.

        Parameters
        ----------
        track_button: ipywidget
            button that when toggled indicates that user is ready to select the model
            tracks for the chosen storm.
        """
        if self.track_button.value is True:
            unique_models = self.NHCD.get_tracks(self.year, self.filename)
            self.model_select = widgets.SelectMultiple(options=unique_models,
                                                       value=[unique_models[0]],
                                                       description='Models: ',
                                                       disabled=False)
            widgets.interact(self.get_models, models=self.model_select)

    def get_models(self, models):
        """
        Select models from a list of all models for a given stormself.

        Parameters
        ----------
        models: list
            list of ran for a given storm
        """
        self.NHCD.model_selection_latlon(models)
        self.date_times = self.NHCD.date_times.tolist()
        self.plot_slider = widgets.IntSlider(min=0, max=(len(self.date_times)-1),
                                                 value=0, description='Tracks Time',
                                                 disabled=False)
        self.play = widgets.Play(interval=800, min=0, max=(len(self.date_times)-1),
                                                 value=0, description='Tracks Time')
        widgets.jslink((self.plot_slider, 'value'), (self.play, 'value'))
        self.box = widgets.HBox([self.plot_slider, self.play])
        display(self.plot_slider)
        widgets.interact(self.plotting, plot_slider=self.play)

    def plotting(self, plot_slider):
        """
        Plot selected model tracks and best track for given storm.

        Parameters
        ----------
        plot_slider: ipywidget
            Widget where assigned value is plot/play widget value
        """
        if self.plot_slider.disabled is False:

            # Identifying the time associated with the models for time text box
            year = self.date_times[plot_slider][0: 4]
            month = self.date_times[plot_slider][4: 6]
            day = self.date_times[plot_slider][6: 8]
            hour = self.date_times[plot_slider][8: 10]
            time_string = 'Date: {0}/{1}/{2} \nHour: {3}'.format(month, day, year, hour)

            # Finding data for best track, and extremes for which to base axis extent on
            self.best_lats = np.array(self.NHCD.best_track_coordinates[0])
            self.best_lons = np.array(self.NHCD.best_track_coordinates[1])
            min_best_lat = min(self.best_lats)
            max_best_lat = max(self.best_lats)
            min_best_lon = min(self.best_lons)
            max_best_lon = max(self.best_lons)


            #Plotting the tracks on top of a cartopy stock image projection
            current_path = os.getcwd()
            os.environ['CARTOPY_USER_BACKGROUNDS'] = current_path
            self.fig = plt.figure(figsize=(14, 11))
            self.ax = self.fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
            self.ax.background_img(name='BM', resolution='low')

            self.data_projection = ccrs.PlateCarree()
            self.ax.plot(self.best_lons, self.best_lats, marker='o', color='white',
                         label='Best Track', transform=self.data_projection)

            self.ax.set_extent([(min_best_lon - 40), (max_best_lon + 40),
                                (min_best_lat - 40), (max_best_lat + 40)])
            


            jet = plt.get_cmap('jet')
            colors = iter(jet(np.linspace(0.2, 1, (len(self.model_select.value)+1))))
            left = .1
            bottom = .1
            self.ax.text(left, bottom, time_string, transform=self.ax.transAxes,
                         fontsize=14, color='white')

            for model_type in self.NHCD.model_table:
                one_model_time = model_type[model_type['WarnDT'] == self.date_times[plot_slider]]
                lats = one_model_time['Lat'].tolist()
                lons = one_model_time['Lon'].tolist()
                if len(lats) != 0:
                    model_list = model_type['Model'].tolist()
                    self.ax.plot(lons, lats, marker='o', color=next(colors),
                                 label=model_list[0])
            plt.title('Storm Name: {0} Year: {1}'.format(self.storm_names.value,
                                                        self.year))
            plt.legend()

NHC_GUI()


interactive(children=(IntSlider(value=2019, description='Storm Year: ', max=2019, min=1851), Output()), _dom_c…

interactive(children=(ToggleButton(value=False, button_style='info', description='Get Storm Tracks', tooltip='…